# P&L Statement

This notebook aggregates all trades between specific time periods, for a breakdown of:

1. Short v Long Term Profits (for tax purposes)
2. Stock v Options Profits (for investment vehicle breakdown)

In [ ]:
import robinhood.logic.dataframe

all_trades = robinhood.logic.dataframe.trades.get(
    from_date='2020-01-01',
    to_date='2020-12-31',
)

## Overview

In [ ]:
import pandas as pd
from IPython.display import display


def get_overview_dataframe(trades: pd.DataFrame):
    trade_types = trades.apply(
        lambda row: 'Options' if len(row['Name']) > 6 else 'Stocks',
        axis=1,
    )
    trade_types.name = 'Type'

    durations = trades.apply(
        lambda row: (
            'Short'
            if (row['Date Sold'] - row['Date Bought']).days < 365
            else 'Long'
        ),
        axis=1,
    )
    durations.name = 'Duration'

    data = pd.concat([trade_types, durations, trades['Earnings']], axis=1)
    return data.groupby(['Type', 'Duration']).sum()


print(
    'Total: ${:0,.2f}'.format(
        get_overview_dataframe(all_trades).sum()[0]
    )
)
display(get_overview_dataframe(all_trades))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


def display_overview_pie_chart(trades: pd.DataFrame):
    data = get_overview_dataframe(trades)

    def format_label(row):
        if row.name[0] == 'Stocks':
            return f'{row.name[0]} ({row.name[1]})'
        else:
            return row.name[0] 

    # Format graph
    figure, ax = plt.subplots()
    # figure.set_size_inches(8, 8)
    ax.pie(
        data['Earnings'],
        labels=data.apply(format_label, axis=1),
        autopct='%1.1f%%',
        shadow=True,
    )
    ax.axis('equal')    # ensures pie is drawn as a circle

    plt.show()


display_overview_pie_chart(all_trades)

In [ ]:
import pandas as pd


def display_monthly_earnings_chart(trades: pd.DataFrame):
    month_index = trades.apply(lambda row: row['Date Sold'].month, axis=1)
    month_index.name = 'Month'

    data = (
        pd.concat([month_index, trades['Earnings']], axis=1)
        .groupby('Month').sum()
    )

    data.plot.bar()


display_monthly_earnings_chart(all_trades)

In [ ]:
# TODO: ROI?
# TODO: Expected profit after tax?

## Stock Trades

In [ ]:
from IPython.display import display
from IPython.display import HTML

display(
    HTML(
        all_trades[all_trades.apply(lambda row: len(row['Name']) <= 6, axis=1)]
        .sort_values(by=['Date Sold', 'Name'])
        .to_html(index=False)
    )
)

In [ ]:
# TODO: Distribution of profit?
# TODO: Largest losses?

## Options Trades

In [ ]:
import re

from IPython.display import display
from IPython.display import HTML
import pandas as pd


def get_option_trades(trades: pd.DataFrame):
    option_trades = all_trades[
        all_trades.apply(lambda row: len(row['Name']) > 6, axis=1)
    ]

    ticker_name_regex = re.compile(r'^([A-Z]+)\d{6}')
    option_trades['Name'] = option_trades.apply(
        lambda row: ticker_name_regex.match(row['Name']).group(1),
        axis=1,
    )

    return option_trades


display(
    HTML(
        get_option_trades(all_trades)
        .sort_values(by=['Date Sold', 'Name'])
        .to_html(index=False)
    )
)

### Most Profitable Trades

In [ ]:
import pandas as pd


def get_most_profitable_option_trades(trades: pd.DataFrame):
    data = pd.concat(
        [
            trades['Name'],
            trades['Earnings']
        ],
        axis=1,
    )

    display(
        data.groupby(['Name']).sum()
        .sort_values('Earnings', ascending=False)
    )


get_most_profitable_option_trades(get_option_trades(all_trades))